# Build Declarative Pipelines using Snowflake Dynamic Tables

## Overview

This guide demonstrates how to build a declarative incremental pipeline and establish a multi-layered lakehouse architecture in Snowflake using **Dynamic Tables** with native Snowflake tables.

Unlike the original tutorial that uses Iceberg tables with external AWS Glue Catalog integration, this simplified version focuses purely on Snowflake's native capabilities, making it easier to get started with declarative data pipelines.

### What You Will Learn

By the end of this guide, you will learn to:
* Create a schema for organizing your data pipeline
* Implement Dynamic Tables with automated refresh using TARGET_LAG
* Model a multi-layered lakehouse: Bronze (raw data), Silver (cleaned and enhanced data), and Gold (aggregated, analytical data)
* Configure incremental refresh modes for efficient data processing
* Verify how incremental data inserts into the Bronze layer automatically propagate through the Silver and Gold layers

### What You'll Build

You will build a three-tiered lakehouse architecture in a single Snowflake schema:
* **Bronze Layer**: Raw transactional data stored in regular Snowflake tables
* **Silver Layer**: Dynamic Tables that apply cleaning and standardization to Bronze data
* **Gold Layer**: Dynamic Tables with denormalized and aggregated data optimized for analytical reporting

All layers will reside in: **MASTERCLASS.02_declarative_pipelines**

### Prerequisites

* A Snowflake account with necessary permissions to create schemas and Dynamic Tables
* An existing MASTERCLASS database
* An existing warehouse for compute
* Basic understanding of SQL
* Familiarity with data warehouse concepts

### Architecture Diagram

```
┌──────────────────────────────────────────────────────────────┐
│          MASTERCLASS.02_declarative_pipelines                │
│                                                              │
│  ┌────────────────────────────────────────────────────────┐ │
│  │              BRONZE LAYER (Regular Tables)             │ │
│  │  de_orders | de_order_items | de_products | de_customers│ │
│  └─────────────────────┬──────────────────────────────────┘ │
│                        │                                     │
│            Dynamic Tables (TARGET_LAG: 1 minute)            │
│                        ↓                                     │
│  ┌────────────────────────────────────────────────────────┐ │
│  │           SILVER LAYER (Dynamic Tables)                │ │
│  │      de_orders_cleaned | de_order_items_enriched       │ │
│  └─────────────────────┬──────────────────────────────────┘ │
│                        │                                     │
│            Dynamic Tables (TARGET_LAG: 1 minute)            │
│                        ↓                                     │
│  ┌────────────────────────────────────────────────────────┐ │
│  │            GOLD LAYER (Dynamic Tables)                 │ │
│  │         order_summary | sales_summary_trends           │ │
│  └────────────────────────────────────────────────────────┘ │
└──────────────────────────────────────────────────────────────┘
```


## Step 1: Schema Setup

In this section, we'll set up the schema for our data pipeline. All Bronze, Silver, and Gold layer tables will reside in the same schema: **MASTERCLASS.02_declarative_pipelines**

### Create Schema


In [ ]:
-- Use the MASTERCLASS database
USE DATABASE MASTERCLASS;

-- Create the schema for our declarative pipeline
CREATE SCHEMA IF NOT EXISTS "02_declarative_pipelines";

-- Use the schema
USE SCHEMA "02_declarative_pipelines";

-- Verify schema was created
SHOW SCHEMAS LIKE '02_declarative_pipelines' IN DATABASE MASTERCLASS;


## Step 2: Bronze Layer - Raw Data Tables

The Bronze layer contains raw, unprocessed data stored in regular Snowflake tables. This layer serves as the foundation for our data pipeline.

We'll create four tables:
* **de_orders**: Order transaction data
* **de_order_items**: Line items for each order
* **de_products**: Product dimension data
* **de_customers**: Customer dimension data

### Create Bronze Layer Tables


In [ ]:
-- Use MASTERCLASS database and schema
USE DATABASE MASTERCLASS;
USE SCHEMA "02_declarative_pipelines";

-- Create de_orders table
CREATE OR REPLACE TABLE de_orders (
    billing_address STRING,
    created_at TIMESTAMP,
    currency STRING,
    customer_id NUMBER(38,0),
    delivery_date TIMESTAMP,
    discount_amount FLOAT,
    notes STRING,
    order_date TIMESTAMP,
    order_id NUMBER(38,0),
    order_status STRING,
    order_uuid STRING,
    payment_method STRING,
    payment_status STRING,
    shipping_address STRING,
    shipping_cost FLOAT,
    shipping_date TIMESTAMP,
    shipping_method STRING,
    subtotal FLOAT,
    tax_amount FLOAT,
    total_amount FLOAT,
    updated_at TIMESTAMP
);

-- Create de_order_items table
CREATE OR REPLACE TABLE de_order_items (
    created_at TIMESTAMP,
    discount_percent NUMBER(38,0),
    line_total FLOAT,
    order_id NUMBER(38,0),
    order_item_id NUMBER(38,0),
    product_id NUMBER(38,0),
    quantity NUMBER(38,0),
    tax_rate FLOAT,
    total_price FLOAT,
    unit_price FLOAT,
    updated_at TIMESTAMP
);

-- Create de_products table
CREATE OR REPLACE TABLE de_products (
    barcode STRING,
    brand STRING,
    category STRING,
    color STRING,
    cost_price FLOAT,
    created_at TIMESTAMP,
    description STRING,
    dimensions_cm STRING,
    is_active BOOLEAN,
    launch_date TIMESTAMP,
    material STRING,
    product_id NUMBER(38,0),
    product_name STRING,
    product_uuid STRING,
    reorder_level NUMBER(38,0),
    size STRING,
    sku STRING,
    stock_quantity NUMBER(38,0),
    subcategory STRING,
    supplier_id NUMBER(38,0),
    unit_price FLOAT,
    updated_at TIMESTAMP,
    weight_kg FLOAT
);

-- Create de_customers table
CREATE OR REPLACE TABLE de_customers (
    address_line1 STRING,
    address_line2 STRING,
    city STRING,
    country STRING,
    created_at TIMESTAMP,
    customer_id NUMBER(38,0),
    customer_segment STRING,
    customer_uuid STRING,
    date_of_birth TIMESTAMP,
    email STRING,
    first_name STRING,
    gender STRING,
    is_active BOOLEAN,
    last_login_date TIMESTAMP,
    last_name STRING,
    phone STRING,
    postal_code STRING,
    registration_date TIMESTAMP,
    state STRING,
    total_orders NUMBER(38,0),
    total_spent FLOAT,
    updated_at TIMESTAMP
);

-- Verify tables were created
SHOW TABLES IN SCHEMA MASTERCLASS."02_declarative_pipelines";


In [ ]:
-- Insert sample customers
INSERT INTO de_customers VALUES
('123 Main St', 'Apt 4B', 'New York', 'USA', '2024-01-15 10:00:00', 1001, 'Premium', 'CUST-UUID-1001', '1985-05-15', 'john.doe@email.com', 'John', 'M', TRUE, '2024-12-10', 'Doe', '555-0101', '10001', '2024-01-15', 'NY', 5, 1250.50, '2024-12-10'),
('456 Oak Ave', NULL, 'Los Angeles', 'USA', '2024-02-20 11:30:00', 1002, 'Standard', 'CUST-UUID-1002', '1990-08-22', 'jane.smith@email.com', 'Jane', 'F', TRUE, '2024-12-08', 'Smith', '555-0102', '90001', '2024-02-20', 'CA', 3, 890.75, '2024-12-08'),
('789 Pine Rd', 'Unit 12', 'Chicago', 'USA', '2024-03-10 14:15:00', 1003, 'Premium', 'CUST-UUID-1003', '1988-11-30', 'bob.johnson@email.com', 'Bob', 'M', TRUE, '2024-12-12', 'Johnson', '555-0103', '60601', '2024-03-10', 'IL', 8, 2150.25, '2024-12-12'),
('321 Elm Dr', NULL, 'Houston', 'USA', '2024-04-05 09:45:00', 1004, 'Standard', 'CUST-UUID-1004', '1992-03-18', 'alice.williams@email.com', 'Alice', 'F', TRUE, '2024-12-11', 'Williams', '555-0104', '77001', '2024-04-05', 'TX', 4, 675.90, '2024-12-11'),
('654 Maple Ln', 'Suite 200', 'Phoenix', 'USA', '2024-05-12 16:20:00', 1005, 'Premium', 'CUST-UUID-1005', '1987-07-25', 'charlie.brown@email.com', 'Charlie', 'M', TRUE, '2024-12-09', 'Brown', '555-0105', '85001', '2024-05-12', 'AZ', 6, 1580.40, '2024-12-09');

-- Insert sample products
INSERT INTO de_products VALUES
('1234567890123', 'TechBrand', 'Electronics', 'Black', 50.00, '2024-01-01', 'Wireless Mouse', '10x6x4', TRUE, '2024-01-01', 'Plastic', 3001, 'Wireless Mouse Pro', 'PROD-UUID-3001', 10, 'Standard', 'SKU-WM-001', 150, 'Computer Accessories', 2001, 79.99, '2024-12-01', 0.15),
('1234567890124', 'TechBrand', 'Electronics', 'Silver', 120.00, '2024-01-05', 'Mechanical Keyboard', '45x15x3', TRUE, '2024-01-05', 'Aluminum', 3002, 'Mechanical Keyboard RGB', 'PROD-UUID-3002', 5, 'Full Size', 'SKU-KB-002', 80, 'Computer Accessories', 2001, 149.99, '2024-12-01', 0.85),
('1234567890125', 'HomeBrand', 'Home & Kitchen', 'White', 25.00, '2024-01-10', 'Coffee Maker', '30x20x35', TRUE, '2024-01-10', 'Plastic', 3003, 'Smart Coffee Maker', 'PROD-UUID-3003', 15, 'Large', 'SKU-CM-003', 200, 'Appliances', 2002, 89.99, '2024-12-01', 2.50),
('1234567890126', 'SportsBrand', 'Sports & Outdoors', 'Blue', 15.00, '2024-01-15', 'Yoga Mat', '180x60x0.6', TRUE, '2024-01-15', 'Rubber', 3004, 'Premium Yoga Mat', 'PROD-UUID-3004', 20, 'Standard', 'SKU-YM-004', 300, 'Fitness', 2003, 39.99, '2024-12-01', 1.20),
('1234567890127', 'FashionBrand', 'Clothing', 'Navy', 30.00, '2024-01-20', 'Cotton T-Shirt', 'M', TRUE, '2024-01-20', 'Cotton', 3005, 'Classic Cotton Tee', 'PROD-UUID-3005', 50, 'M', 'SKU-TS-005', 500, 'Casual Wear', 2004, 29.99, '2024-12-01', 0.20);

SELECT 'Loaded ' || COUNT(*) || ' customers' FROM de_customers;


In [ ]:
-- Insert sample orders
INSERT INTO de_orders VALUES
('123 Main St, New York, NY 10001', '2024-12-01 10:30:00', 'USD', 1001, '2024-12-05 14:00:00', 0.00, 'First order', '2024-12-01 10:30:00', 5001, 'delivered', 'ORD-2024-5001', 'credit_card', 'paid', '123 Main St, New York, NY 10001', 9.99, '2024-12-02 08:00:00', 'standard', 79.99, 6.40, 96.38, '2024-12-05 14:00:00'),
('456 Oak Ave, Los Angeles, CA 90001', '2024-12-02 14:15:00', 'USD', 1002, '2024-12-06 16:30:00', 15.00, 'Express delivery', '2024-12-02 14:15:00', 5002, 'delivered', 'ORD-2024-5002', 'paypal', 'paid', '456 Oak Ave, Los Angeles, CA 90001', 19.99, '2024-12-03 09:00:00', 'express', 149.99, 12.00, 166.98, '2024-12-06 16:30:00'),
('789 Pine Rd, Chicago, IL 60601', '2024-12-03 09:45:00', 'USD', 1003, '2024-12-07 11:00:00', 0.00, 'Gift order', '2024-12-03 09:45:00', 5003, 'shipped', 'ORD-2024-5003', 'credit_card', 'paid', '789 Pine Rd, Unit 12, Chicago, IL 60601', 9.99, '2024-12-04 10:00:00', 'standard', 89.99, 7.20, 107.18, '2024-12-04 10:00:00'),
('321 Elm Dr, Houston, TX 77001', '2024-12-04 16:20:00', 'USD', 1004, '2024-12-08 12:00:00', 0.00, 'Standard order', '2024-12-04 16:20:00', 5004, 'processing', 'ORD-2024-5004', 'debit_card', 'paid', '321 Elm Dr, Houston, TX 77001', 9.99, '2024-12-05 11:00:00', 'standard', 39.99, 3.20, 53.18, '2024-12-04 16:20:00'),
('654 Maple Ln, Phoenix, AZ 85001', '2024-12-05 11:00:00', 'USD', 1005, NULL, 0.00, 'Bulk order', '2024-12-05 11:00:00', 5005, 'confirmed', 'ORD-2024-5005', 'credit_card', 'paid', '654 Maple Ln, Suite 200, Phoenix, AZ 85001', 9.99, NULL, 'standard', 119.96, 9.60, 139.55, '2024-12-05 11:00:00'),
('123 Main St, New York, NY 10001', '2024-12-06 13:30:00', 'USD', 1001, '2024-12-10 15:00:00', 10.00, 'Repeat customer discount', '2024-12-06 13:30:00', 5006, 'delivered', 'ORD-2024-5006', 'credit_card', 'paid', '123 Main St, New York, NY 10001', 9.99, '2024-12-07 09:00:00', 'standard', 149.99, 12.00, 161.98, '2024-12-10 15:00:00'),
('789 Pine Rd, Chicago, IL 60601', '2024-12-07 10:15:00', 'USD', 1003, '2024-12-11 16:00:00', 0.00, 'Holiday order', '2024-12-07 10:15:00', 5007, 'delivered', 'ORD-2024-5007', 'paypal', 'paid', '789 Pine Rd, Unit 12, Chicago, IL 60601', 9.99, '2024-12-08 08:00:00', 'standard', 69.98, 5.60, 85.57, '2024-12-11 16:00:00'),
('456 Oak Ave, Los Angeles, CA 90001', '2024-12-08 15:45:00', 'USD', 1002, '2024-12-12 14:00:00', 0.00, 'Standard order', '2024-12-08 15:45:00', 5008, 'shipped', 'ORD-2024-5008', 'credit_card', 'paid', '456 Oak Ave, Los Angeles, CA 90001', 9.99, '2024-12-09 10:00:00', 'standard', 89.99, 7.20, 107.18, '2024-12-09 10:00:00'),
('654 Maple Ln, Phoenix, AZ 85001', '2024-12-09 12:00:00', 'USD', 1005, NULL, 5.00, 'Quick reorder', '2024-12-09 12:00:00', 5009, 'processing', 'ORD-2024-5009', 'debit_card', 'paid', '654 Maple Ln, Suite 200, Phoenix, AZ 85001', 9.99, NULL, 'standard', 79.99, 6.40, 91.38, '2024-12-09 12:00:00'),
('321 Elm Dr, Houston, TX 77001', '2024-12-10 09:30:00', 'USD', 1004, NULL, 0.00, 'New order', '2024-12-10 09:30:00', 5010, 'confirmed', 'ORD-2024-5010', 'credit_card', 'pending', '321 Elm Dr, Houston, TX 77001', 9.99, NULL, 'standard', 29.99, 2.40, 42.38, '2024-12-10 09:30:00');

-- Insert sample order items
INSERT INTO de_order_items VALUES
('2024-12-01 10:30:00', 0, 79.99, 5001, 10001, 3001, 1, 0.08, 79.99, 79.99, '2024-12-01 10:30:00'),
('2024-12-02 14:15:00', 10, 149.99, 5002, 10002, 3002, 1, 0.08, 134.99, 149.99, '2024-12-02 14:15:00'),
('2024-12-03 09:45:00', 0, 89.99, 5003, 10003, 3003, 1, 0.08, 89.99, 89.99, '2024-12-03 09:45:00'),
('2024-12-04 16:20:00', 0, 39.99, 5004, 10004, 3004, 1, 0.08, 39.99, 39.99, '2024-12-04 16:20:00'),
('2024-12-05 11:00:00', 0, 119.96, 5005, 10005, 3005, 4, 0.08, 119.96, 29.99, '2024-12-05 11:00:00'),
('2024-12-06 13:30:00', 0, 149.99, 5006, 10006, 3002, 1, 0.08, 149.99, 149.99, '2024-12-06 13:30:00'),
('2024-12-07 10:15:00', 0, 69.98, 5007, 10007, 3004, 2, 0.08, 69.98, 39.99, '2024-12-07 10:15:00'),
('2024-12-08 15:45:00', 0, 89.99, 5008, 10008, 3003, 1, 0.08, 89.99, 89.99, '2024-12-08 15:45:00'),
('2024-12-09 12:00:00', 0, 79.99, 5009, 10009, 3001, 1, 0.08, 79.99, 79.99, '2024-12-09 12:00:00'),
('2024-12-10 09:30:00', 0, 29.99, 5010, 10010, 3005, 1, 0.08, 29.99, 29.99, '2024-12-10 09:30:00');

SELECT 'Loaded ' || COUNT(*) || ' orders' FROM de_orders;


### Verify Bronze Layer Data

Let's check the row counts in our Bronze layer tables.


In [ ]:
-- Check Bronze layer row counts
SELECT 'de_customers' AS table_name, COUNT(*) AS row_count FROM de_customers
UNION ALL
SELECT 'de_products', COUNT(*) FROM de_products
UNION ALL
SELECT 'de_orders', COUNT(*) FROM de_orders
UNION ALL
SELECT 'de_order_items', COUNT(*) FROM de_order_items
ORDER BY table_name;


## Step 3: Silver Layer - Dynamic Tables for Data Transformation

The Silver layer uses **Dynamic Tables** to automatically clean, transform, and enrich data from the Bronze layer. Dynamic Tables are declarative - you define the query, and Snowflake handles the refresh automatically.

### Key Concepts:
* **TARGET_LAG**: Defines how fresh the data should be (e.g., '1 minute' means data will be refreshed within 1 minute of changes)
* **REFRESH_MODE**: Can be INCREMENTAL (process only new/changed data) or FULL (reprocess all data)
* **WAREHOUSE**: The compute warehouse used for refreshing the dynamic table

### Create Silver Layer Dynamic Tables


In [ ]:
-- Ensure we're using the correct database and schema
USE DATABASE MASTERCLASS;
USE SCHEMA "02_declarative_pipelines";

-- Create de_orders_cleaned Dynamic Table
-- This table cleans and standardizes orders data
CREATE OR REPLACE DYNAMIC TABLE de_orders_cleaned
TARGET_LAG = '1 minute'
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = INCREMENTAL
AS
SELECT 
    order_id,
    order_uuid,
    customer_id,
    order_date,
    order_status,
    UPPER(TRIM(payment_method)) AS payment_method_std,
    UPPER(TRIM(payment_status)) AS payment_status_std,
    UPPER(TRIM(shipping_method)) AS shipping_method_std,
    currency,
    subtotal,
    discount_amount,
    tax_amount,
    shipping_cost,
    total_amount,
    billing_address,
    shipping_address,
    shipping_date,
    delivery_date,
    notes,
    created_at,
    updated_at,
    -- Calculated fields
    DATEDIFF(day, order_date, shipping_date) AS days_to_ship,
    DATEDIFF(day, shipping_date, delivery_date) AS days_to_deliver,
    CASE 
        WHEN order_status = 'delivered' THEN 1
        WHEN order_status = 'shipped' THEN 0.75
        WHEN order_status = 'processing' THEN 0.5
        WHEN order_status = 'confirmed' THEN 0.25
        ELSE 0
    END AS order_progress_score
FROM MASTERCLASS."02_declarative_pipelines".de_orders
WHERE order_date IS NOT NULL;

-- Verify the dynamic table was created
SHOW DYNAMIC TABLES LIKE 'de_orders_cleaned';


In [ ]:
-- Create de_order_items_enriched Dynamic Table
-- This table joins order items with product information
CREATE OR REPLACE DYNAMIC TABLE de_order_items_enriched
TARGET_LAG = '1 minute'
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = INCREMENTAL
AS
SELECT 
    oi.order_item_id,
    oi.order_id,
    oi.product_id,
    oi.quantity,
    oi.unit_price,
    oi.discount_percent,
    oi.tax_rate,
    oi.line_total,
    oi.total_price,
    -- Product enrichment
    p.product_name,
    p.sku,
    p.brand,
    p.category,
    p.subcategory,
    p.color,
    p.size,
    p.material,
    p.cost_price,
    -- Calculated fields
    (oi.unit_price - p.cost_price) AS profit_per_unit,
    (oi.total_price - (p.cost_price * oi.quantity)) AS profit_per_line,
    ROUND((oi.unit_price - p.cost_price) / NULLIF(p.cost_price, 0) * 100, 2) AS profit_margin_percent,
    oi.created_at,
    oi.updated_at
FROM MASTERCLASS."02_declarative_pipelines".de_order_items oi
INNER JOIN MASTERCLASS."02_declarative_pipelines".de_products p
    ON oi.product_id = p.product_id
WHERE oi.order_id IS NOT NULL;

-- Verify the dynamic table was created
SHOW DYNAMIC TABLES LIKE 'de_order_items_enriched';


### Wait for Initial Refresh and Verify Silver Layer

Dynamic Tables refresh automatically based on TARGET_LAG. Let's wait a moment and then check the data.


In [ ]:
-- Check Silver layer row counts
SELECT 'de_orders_cleaned' AS table_name, COUNT(*) AS row_count FROM de_orders_cleaned
UNION ALL
SELECT 'de_order_items_enriched', COUNT(*) FROM de_order_items_enriched
ORDER BY table_name;

-- Sample cleaned orders data
SELECT * FROM de_orders_cleaned LIMIT 5;

-- Sample enriched order items with profit calculations
SELECT 
    order_item_id,
    order_id,
    product_name,
    brand,
    category,
    quantity,
    unit_price,
    cost_price,
    profit_per_unit,
    profit_margin_percent
FROM de_order_items_enriched
LIMIT 5;


## Step 4: Gold Layer - Aggregated Analytics with Dynamic Tables

The Gold layer contains business-ready, aggregated data optimized for analytics and reporting. These Dynamic Tables join and aggregate data from the Silver layer.

### Create Gold Layer Dynamic Tables


In [ ]:
-- Ensure we're using the correct database and schema
USE DATABASE MASTERCLASS;
USE SCHEMA "02_declarative_pipelines";

-- Create order_summary Dynamic Table
-- This table provides a complete view of each order with customer and item details
CREATE OR REPLACE DYNAMIC TABLE order_summary
TARGET_LAG = '1 minute'
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = INCREMENTAL
AS
SELECT 
    o.order_id,
    o.order_uuid,
    o.order_date,
    o.order_status,
    -- Customer information
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.customer_segment,
    c.email AS customer_email,
    c.city,
    c.state,
    c.country,
    -- Order financials
    o.subtotal,
    o.discount_amount,
    o.tax_amount,
    o.shipping_cost,
    o.total_amount,
    o.payment_method_std AS payment_method,
    o.payment_status_std AS payment_status,
    -- Shipping details
    o.shipping_method_std AS shipping_method,
    o.shipping_date,
    o.delivery_date,
    o.days_to_ship,
    o.days_to_deliver,
    -- Aggregated order item metrics
    COUNT(oi.order_item_id) AS total_items,
    SUM(oi.quantity) AS total_quantity,
    SUM(oi.profit_per_line) AS total_profit,
    ROUND(AVG(oi.profit_margin_percent), 2) AS avg_profit_margin_percent,
    -- Order categorization
    CASE 
        WHEN o.total_amount >= 150 THEN 'High Value'
        WHEN o.total_amount >= 75 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS order_value_category,
    o.created_at,
    o.updated_at
FROM MASTERCLASS."02_declarative_pipelines".de_orders_cleaned o
INNER JOIN MASTERCLASS."02_declarative_pipelines".de_customers c
    ON o.customer_id = c.customer_id
LEFT JOIN MASTERCLASS."02_declarative_pipelines".de_order_items_enriched oi
    ON o.order_id = oi.order_id
GROUP BY 
    o.order_id, o.order_uuid, o.order_date, o.order_status,
    c.customer_id, c.first_name, c.last_name, c.customer_segment,
    c.email, c.city, c.state, c.country,
    o.subtotal, o.discount_amount, o.tax_amount, o.shipping_cost,
    o.total_amount, o.payment_method_std, o.payment_status_std,
    o.shipping_method_std, o.shipping_date, o.delivery_date,
    o.days_to_ship, o.days_to_deliver,
    o.created_at, o.updated_at;

-- Verify the dynamic table was created
SHOW DYNAMIC TABLES LIKE 'order_summary';


In [ ]:
-- Create sales_summary_trends Dynamic Table
-- This table provides time-based sales analytics
CREATE OR REPLACE DYNAMIC TABLE sales_summary_trends
TARGET_LAG = '1 minute'
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = INCREMENTAL
AS
SELECT 
    DATE_TRUNC('day', o.order_date) AS order_day,
    -- Order counts
    COUNT(DISTINCT o.order_id) AS total_orders,
    COUNT(DISTINCT o.customer_id) AS unique_customers,
    -- Revenue metrics
    SUM(o.total_amount) AS total_revenue,
    SUM(o.subtotal) AS total_subtotal,
    SUM(o.discount_amount) AS total_discounts,
    SUM(o.tax_amount) AS total_tax,
    SUM(o.shipping_cost) AS total_shipping,
    AVG(o.total_amount) AS avg_order_value,
    -- Profitability metrics
    SUM(oi.profit_per_line) AS total_profit,
    ROUND(AVG(oi.profit_margin_percent), 2) AS avg_profit_margin,
    -- Product metrics
    COUNT(DISTINCT oi.product_id) AS unique_products_sold,
    SUM(oi.quantity) AS total_units_sold,
    -- Category breakdown
    COUNT(DISTINCT CASE WHEN oi.category = 'Electronics' THEN oi.order_item_id END) AS electronics_items,
    COUNT(DISTINCT CASE WHEN oi.category = 'Home & Kitchen' THEN oi.order_item_id END) AS home_items,
    COUNT(DISTINCT CASE WHEN oi.category = 'Sports & Outdoors' THEN oi.order_item_id END) AS sports_items,
    COUNT(DISTINCT CASE WHEN oi.category = 'Clothing' THEN oi.order_item_id END) AS clothing_items,
    -- Payment method breakdown
    COUNT(DISTINCT CASE WHEN o.payment_method_std = 'CREDIT_CARD' THEN o.order_id END) AS credit_card_orders,
    COUNT(DISTINCT CASE WHEN o.payment_method_std = 'PAYPAL' THEN o.order_id END) AS paypal_orders,
    COUNT(DISTINCT CASE WHEN o.payment_method_std = 'DEBIT_CARD' THEN o.order_id END) AS debit_card_orders,
    -- Order status breakdown
    COUNT(DISTINCT CASE WHEN o.order_status = 'delivered' THEN o.order_id END) AS delivered_orders,
    COUNT(DISTINCT CASE WHEN o.order_status = 'shipped' THEN o.order_id END) AS shipped_orders,
    COUNT(DISTINCT CASE WHEN o.order_status = 'processing' THEN o.order_id END) AS processing_orders,
    COUNT(DISTINCT CASE WHEN o.order_status = 'confirmed' THEN o.order_id END) AS confirmed_orders
FROM MASTERCLASS."02_declarative_pipelines".de_orders_cleaned o
LEFT JOIN MASTERCLASS."02_declarative_pipelines".de_order_items_enriched oi
    ON o.order_id = oi.order_id
GROUP BY DATE_TRUNC('day', o.order_date)
ORDER BY order_day;

-- Verify the dynamic table was created
SHOW DYNAMIC TABLES LIKE 'sales_summary_trends';


### Verify Gold Layer Data

Let's check the aggregated data in our Gold layer tables.


In [ ]:
-- Check Gold layer row counts
SELECT 'order_summary' AS table_name, COUNT(*) AS row_count FROM order_summary
UNION ALL
SELECT 'sales_summary_trends', COUNT(*) FROM sales_summary_trends
ORDER BY table_name;

-- Sample order summary data with profit calculations
SELECT 
    order_id,
    order_date,
    customer_name,
    customer_segment,
    total_amount,
    total_profit,
    avg_profit_margin_percent,
    order_value_category,
    total_items,
    order_status
FROM order_summary
ORDER BY order_date DESC
LIMIT 5;

-- Sales trends by day
SELECT 
    order_day,
    total_orders,
    unique_customers,
    total_revenue,
    total_profit,
    avg_profit_margin,
    avg_order_value,
    total_units_sold
FROM sales_summary_trends
ORDER BY order_day DESC;


## Step 5: Testing Incremental Updates

Now let's demonstrate the power of Dynamic Tables by inserting new data into the Bronze layer and observing how it automatically propagates through Silver and Gold layers.

### Record Current State

First, let's record the current row counts across all layers.


In [ ]:
-- Record current state across all layers
SELECT 'BRONZE - de_orders' AS layer_table, COUNT(*) AS row_count FROM MASTERCLASS."02_declarative_pipelines".de_orders
UNION ALL
SELECT 'BRONZE - de_order_items', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".de_order_items
UNION ALL
SELECT 'SILVER - de_orders_cleaned', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".de_orders_cleaned
UNION ALL
SELECT 'SILVER - de_order_items_enriched', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".de_order_items_enriched
UNION ALL
SELECT 'GOLD - order_summary', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".order_summary
UNION ALL
SELECT 'GOLD - sales_summary_trends', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".sales_summary_trends
ORDER BY layer_table;


### Insert New Data into Bronze Layer

Let's insert 3 new orders with their associated order items to simulate new transactions.


In [ ]:
-- Use MASTERCLASS database and schema
USE DATABASE MASTERCLASS;
USE SCHEMA "02_declarative_pipelines";

-- Insert 3 new orders
INSERT INTO de_orders VALUES
('123 Main St, New York, NY 10001', '2024-12-15 10:00:00', 'USD', 1001, NULL, 0.00, 'New test order 1', '2024-12-15 10:00:00', 5011, 'confirmed', 'ORD-2024-5011', 'credit_card', 'paid', '123 Main St, New York, NY 10001', 9.99, NULL, 'standard', 149.99, 12.00, 171.98, '2024-12-15 10:00:00'),
('456 Oak Ave, Los Angeles, CA 90001', '2024-12-15 11:30:00', 'USD', 1002, NULL, 20.00, 'New test order 2 with discount', '2024-12-15 11:30:00', 5012, 'processing', 'ORD-2024-5012', 'paypal', 'paid', '456 Oak Ave, Los Angeles, CA 90001', 19.99, NULL, 'express', 299.98, 24.00, 323.97, '2024-12-15 11:30:00'),
('789 Pine Rd, Chicago, IL 60601', '2024-12-15 14:15:00', 'USD', 1003, NULL, 0.00, 'New test order 3', '2024-12-15 14:15:00', 5013, 'confirmed', 'ORD-2024-5013', 'credit_card', 'paid', '789 Pine Rd, Unit 12, Chicago, IL 60601', 9.99, NULL, 'standard', 129.98, 10.40, 150.37, '2024-12-15 14:15:00');

-- Insert corresponding order items
INSERT INTO de_order_items VALUES
('2024-12-15 10:00:00', 0, 149.99, 5011, 10011, 3002, 1, 0.08, 149.99, 149.99, '2024-12-15 10:00:00'),
('2024-12-15 11:30:00', 10, 149.99, 5012, 10012, 3002, 1, 0.08, 134.99, 149.99, '2024-12-15 11:30:00'),
('2024-12-15 11:30:00', 0, 149.99, 5012, 10013, 3002, 1, 0.08, 149.99, 149.99, '2024-12-15 11:30:00'),
('2024-12-15 14:15:00', 0, 89.99, 5013, 10014, 3003, 1, 0.08, 89.99, 89.99, '2024-12-15 14:15:00'),
('2024-12-15 14:15:00', 0, 39.99, 5013, 10015, 3004, 1, 0.08, 39.99, 39.99, '2024-12-15 14:15:00');

SELECT 'Inserted ' || COUNT(*) || ' new orders' AS result
FROM de_orders
WHERE order_id >= 5011;


### Wait for Dynamic Table Refresh

The Dynamic Tables are configured with TARGET_LAG = '1 minute', which means they will automatically refresh within 1 minute of detecting changes in the source tables.

**Important**: In a real scenario, you would wait 1-2 minutes for the automatic refresh. For this tutorial, you can either:
1. Wait for the automatic refresh (recommended)
2. Manually trigger a refresh using `ALTER DYNAMIC TABLE ... REFRESH`

Let's manually trigger the refreshes to see immediate results.


In [ ]:
-- Manually trigger refresh of Silver layer (optional - would happen automatically)
ALTER DYNAMIC TABLE MASTERCLASS."02_declarative_pipelines".de_orders_cleaned REFRESH;
ALTER DYNAMIC TABLE MASTERCLASS."02_declarative_pipelines".de_order_items_enriched REFRESH;

-- Manually trigger refresh of Gold layer (optional - would happen automatically)
ALTER DYNAMIC TABLE MASTERCLASS."02_declarative_pipelines".order_summary REFRESH;
ALTER DYNAMIC TABLE MASTERCLASS."02_declarative_pipelines".sales_summary_trends REFRESH;

SELECT 'Dynamic tables refreshed' AS status;


### Verify Data Propagation

Now let's check that the new data has propagated through all layers.


In [ ]:
-- Check updated row counts after incremental insert
SELECT 'BRONZE - de_orders' AS layer_table, COUNT(*) AS row_count FROM MASTERCLASS."02_declarative_pipelines".de_orders
UNION ALL
SELECT 'BRONZE - de_order_items', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".de_order_items
UNION ALL
SELECT 'SILVER - de_orders_cleaned', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".de_orders_cleaned
UNION ALL
SELECT 'SILVER - de_order_items_enriched', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".de_order_items_enriched
UNION ALL
SELECT 'GOLD - order_summary', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".order_summary
UNION ALL
SELECT 'GOLD - sales_summary_trends', COUNT(*) FROM MASTERCLASS."02_declarative_pipelines".sales_summary_trends
ORDER BY layer_table;

-- View the newly added orders in the Gold layer
SELECT 
    order_id,
    order_date,
    customer_name,
    total_amount,
    total_profit,
    order_value_category,
    order_status
FROM MASTERCLASS."02_declarative_pipelines".order_summary
WHERE order_id >= 5011
ORDER BY order_id;


## Step 6: Monitoring Dynamic Tables

Snowflake provides system views to monitor the status and refresh history of Dynamic Tables. Let's explore these monitoring capabilities.


In [ ]:
-- View all Dynamic Tables in the account
SHOW DYNAMIC TABLES;

-- Get detailed information about Dynamic Tables
SELECT 
    name,
    database_name,
    schema_name,
    target_lag,
    refresh_mode,
    warehouse,
    scheduling_state
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLES())
WHERE database_name = 'MASTERCLASS' AND schema_name = '02_declarative_pipelines'
ORDER BY name;


In [ ]:
-- Check refresh history for a specific dynamic table
-- This shows when refreshes occurred and how long they took
SELECT 
    name,
    refresh_start_time,
    refresh_end_time,
    DATEDIFF(second, refresh_start_time, refresh_end_time) AS refresh_duration_seconds,
    state,
    refresh_action
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
    NAME => 'MASTERCLASS."02_declarative_pipelines".order_summary'
))
ORDER BY refresh_start_time DESC
LIMIT 10;


### Check Data Freshness

Monitor data freshness (lag) for Dynamic Tables to ensure they're meeting their TARGET_LAG commitments.


In [ ]:
-- Check the freshness/lag of Dynamic Tables
SELECT 
    CONCAT(database_name, '.', schema_name, '.', name) AS full_table_name,
    target_lag,
    data_timestamp,
    scheduling_state,
    CASE 
        WHEN scheduling_state = 'RUNNING' THEN 'Actively refreshing'
        WHEN scheduling_state = 'SCHEDULED' THEN 'Scheduled for refresh'
        ELSE scheduling_state
    END AS status_description
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLES())
WHERE database_name = 'MASTERCLASS' AND schema_name = '02_declarative_pipelines'
ORDER BY name;


## Conclusion

Congratulations! You've successfully built a complete declarative data pipeline using Snowflake Dynamic Tables.

### What You Accomplished

In this tutorial, you:

1. **Created a Single-Schema Architecture**: Set up MASTERCLASS."02_declarative_pipelines" schema with Bronze (raw), Silver (transformed), and Gold (aggregated) layers
2. **Implemented Dynamic Tables**: Created automated data pipelines with TARGET_LAG configuration
3. **Configured Incremental Refresh**: Used INCREMENTAL refresh mode for efficient data processing
4. **Tested Data Propagation**: Verified that changes in Bronze automatically flow through Silver to Gold
5. **Monitored Pipeline Health**: Explored system views to track refresh status and data freshness

### Key Benefits of This Approach

**Simplicity**: This approach:
- Requires no external catalog integration
- Eliminates complex S3 permissions and external volumes
- Uses only native Snowflake features
- Keeps all tables in a single, organized schema

**Declarative**: You define WHAT you want (the query), not HOW to maintain it. Snowflake handles:
- Automatic refresh scheduling
- Incremental processing
- Dependency management between layers

**Efficiency**: Dynamic Tables with INCREMENTAL mode:
- Process only new or changed data
- Reduce compute costs
- Minimize latency from source to analytics

### Best Practices

1. **Choose Appropriate TARGET_LAG**: Balance freshness needs with compute costs
   - Near real-time: '1 minute' or '5 minutes'
   - Standard: '1 hour' or 'DOWNSTREAM'
   - Batch: '1 day'

2. **Use INCREMENTAL Mode When Possible**: Most efficient for large datasets with incremental changes

3. **Monitor Regularly**: Check refresh history and scheduling state to ensure pipelines are healthy

4. **Layer Your Data**: Bronze → Silver → Gold pattern provides:
   - Separation of concerns
   - Easier debugging
   - Flexibility to add new analytics
   - Clear data lineage

### Next Steps

- **Experiment with Different TARGET_LAG Values**: See how it affects refresh timing
- **Add More Transformations**: Create additional Silver/Gold tables for different business needs
- **Implement Alerting**: Use Snowflake's monitoring features to alert on pipeline issues
- **Scale Your Data**: Test with larger datasets to see incremental refresh efficiency

### Resources

- [Snowflake Dynamic Tables Documentation](https://docs.snowflake.com/en/user-guide/dynamic-tables-about)
- [Dynamic Tables Best Practices](https://docs.snowflake.com/en/user-guide/dynamic-tables-best-practices)
- [Monitoring Dynamic Tables](https://docs.snowflake.com/en/user-guide/dynamic-tables-tasks-manage)

---

**Thank you for completing this tutorial!** You now have a solid foundation for building declarative data pipelines in Snowflake.
